<a href="https://colab.research.google.com/github/Denis04-M/email_classification/blob/main/spam_email_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import os
import glob

filepath1 = '/content/drive/MyDrive/data/enron1/ham/'
filepath2 = '/content/drive/MyDrive/data/enron1/spam/'

emails, labels = [], []

# appending valid emails into emails[]
for filename in glob.glob(os.path.join(filepath1, '*.txt')):
  with open(filename, 'r', encoding='ISO-8859-1') as email_file:
    emails.append(email_file.read())
    labels.append(1) # labeling each valid email as 1

# appending spam emails into emils[]
for filename in glob.glob(os.path.join(filepath2, '*.txt')):
  with open(filename, 'r', encoding='ISO-8859-1') as email_file:
    emails.append(email_file.read())
    labels.append(0) # labeling each spam email as 0

print(len(emails))
print(len(labels))

5172
5172


In [17]:
from nltk.corpus import names
from  nltk.stem import WordNetLemmatizer

people_names = set(names.words())
lemmatizer = WordNetLemmatizer()


# function to remove numbers and special characters
def letters_only(astr):
  return astr.isalpha()

# cleaning the data (removin)
def cleanText(docs):
  cleaned_docs = []
  for doc in docs:
    cleaned_docs.append(" ".join([lemmatizer.lemmatize(word.lower())
      for word in doc.split()
        if letters_only(word) and word not in people_names]))
  return cleaned_docs

cleaned_emails = cleanText(emails)

print(cleaned_emails[0])

tried to get fancy with your address and it came back to me forwarded by lauri a allen hol aepin on pm to daren farmer enron com cc subject daren your rate for meter highlander central point for year starting delivered to equistar channelview is mm that price expires in week on november let me know if you need me to refresh after that time thanks


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

cv = CountVectorizer(stop_words = 'english', max_features=500)

term_docs = cv.fit_transform(cleaned_emails)

def get_label_index(labels):
  labeld_index = defaultdict(list)
  for index, label in enumerate(labels):
    label_index[label].append(index)
  return label_index

label_index = get_label_index(labels)

def get_prior(label_index):
  prior = {label: len(index) for label, index in label_index.items()}
  total_count = sum(prior.values())
  for label in prior:
    prior[label] /= float(total_count)
  return prior

prior = get_prior(label_index)
print(prior)

{1: 0.7099767981438515, 0: 0.2900232018561485}


In [32]:
import numpy as np

def get_likelihood(term_document_matrix, label_index, smoothing = 0):
  likelihood = {}
  for label, index in label_index.items():
    likelihood[label] = term_document_matrix[index, :].sum(axis=0) + smoothing
    likelihood[label] = np.asarray(likelihood[label])[0]
    total_count = likelihood[label].sum()
    likelihood[label] = likelihood[label] / float(total_count)
  return likelihood

smoothing = 1
likelihood = get_likelihood(term_docs, label_index, smoothing)

print(likelihood[0][:5])

[0.00108188 0.00141386 0.00458837 0.00052167 0.00423564]
